In [1]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

In [2]:
import pandas as pd

In [3]:
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
df = pd.DataFrame(data)

In [4]:
df

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Paid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending


Step 1. User: specify tools and query

In [5]:
import json

In [6]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status": df[df.transaction_id == transaction_id].payment_status.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [7]:
status = retrieve_payment_status(df, transaction_id="T1001")
print(status)

{"status": "Paid"}


In [8]:
type(status)

str

In [9]:
def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"date": df[df.transaction_id == transaction_id].payment_date.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [10]:
date = retrieve_payment_date(df, transaction_id="T1002")
print(date)

{"date": "2021-10-06"}


You can outline the function specifications with a JSON schema.

In [11]:
tool_payment_status = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_status",
        "description": "Get payment status of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [12]:
type(tool_payment_status)

dict

In [13]:
tool_payment_date = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_date",
        "description": "Get payment date of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [14]:
type(tool_payment_status)

dict

In [15]:
tools = [tool_payment_status, tool_payment_date]

In [16]:
type(tools)

list

In [17]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

functools

In [18]:
import functools

In [19]:
names_to_functions = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status, df=df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date, df=df),
}

In [20]:
names_to_functions["retrieve_payment_status"](transaction_id="T1001")

'{"status": "Paid"}'

In [21]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

User query

In [22]:
from mistralai.models.chat_completion import ChatMessage

chat_history = [
    ChatMessage(role="user", content="What's the status of my transaction?")
]

Step 2. Model: Generate function arguments

In [ ]:
from mistralai.client import MistralClient
import os

model = "mistral-large-latest"

client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"), endpoint=os.getenv("DLAI_MISTRAL_API_ENDPOINT"))

response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

response

In [ ]:
response.choices[0].message.content

Save the chat history

In [ ]:
chat_history.append(
    ChatMessage(role="assistant", content=response.choices[0].message.content)
)
chat_history.append(ChatMessage(role="user", content="My transaction ID is T1001."))
chat_history

In [ ]:
response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

In [ ]:
response

In [ ]:
response.choices[0].message

In [ ]:
chat_history.append(response.choices[0].message)

Step 3. User: Execute function to obtain tool results: Currently, the user is the one who will execute these functions (the model will not execute these functions on its own).

In [ ]:
tool_function = response.choices[0].message.tool_calls[0].function
print(tool_function)

In [ ]:
tool_function.name

In [ ]:
tool_function.arguments

In [ ]:
#The function arguments are expected to be in a Python dictionary and not a string.
#To make this string into a dictionary, you can use json.loads()

args = json.loads(tool_function.arguments)
print(args)

In [ ]:
import functools
names_to_functions = {
    "retrieve_payment_status": 
      functools.partial(retrieve_payment_status, df=df),

    "retrieve_payment_date": 
      functools.partial(retrieve_payment_date, df=df),
}

In [ ]:
function_result = names_to_functions[tool_function.name](**args)
function_result

In [ ]:
#The output of the function call can be saved as a chat message, with the role "tool".

tool_msg = ChatMessage(role="tool", name=tool_function.name, content=function_result)
chat_history.append(tool_msg)

In [ ]:
chat_history

Step 4. Model: Generate final answer: The model can now reply to the user query, given the information provided by the "tool".

In [ ]:
response = client.chat(model=model, messages=chat_history)
response.choices[0].message.content